# Classification: Flow regime (laminar vs turbulent)

This notebook synthesizes simple pipe-flow cases, computes Reynolds numbers,
trains a logistic regression to classify laminar vs turbulent flow, and visualizes results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
def compute_reynolds(flow_rate, density, viscosity, diameter):
    area = np.pi * (diameter / 2) ** 2
    velocity = flow_rate / area
    Re = density * velocity * diameter / viscosity
    return Re

rng = np.random.RandomState(2)
n = 400
flow_rate = rng.uniform(1e-4, 0.6, size=n)
density = rng.uniform(900.0, 1100.0, size=n)
viscosity = rng.uniform(1e-5, 1e-2, size=n)
diameter = rng.uniform(0.005, 0.2, size=n)
Re = np.array([compute_reynolds(q, rho, mu, D) for q, rho, mu, D in zip(flow_rate, density, viscosity, diameter)])
# create labels by comparing to the median Re — guarantees two classes and balances labels
y = (Re >= np.median(Re)).astype(int)
X = np.vstack([flow_rate, density, viscosity, diameter, Re]).T

In [ ]:
# Simple deterministic baseline: predict using median(Re) threshold on test set (no fitting)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
thresh = np.median(X[:, 4])
preds = (X_test[:, 4] >= thresh).astype(int)
acc = accuracy_score(y_test, preds)
print(f'Median-Re baseline accuracy: {acc:.3f}')
cm = confusion_matrix(y_test, preds, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['low Re','high Re'])
disp.plot()
plt.show()

In [ ]:
plt.hist([Re[y==0], Re[y==1]], bins=40, stacked=True, label=['laminar','turbulent'])
plt.xlabel('Reynolds number')
plt.ylabel('count')
plt.legend()
plt.title('Distribution of Re (synthetic)')
plt.show()